## Viz -1

In [50]:
import pandas as pd
import plotly.graph_objects as go

# Load the NASA GISTEMP dataset (global land + ocean)
df = pd.read_csv("../data/NASA_Data.csv", skiprows=1)
df.columns = df.columns.str.strip()

# Keep relevant columns
df = df[['Year', 'J-D']].dropna()
df['J-D'] = pd.to_numeric(df['J-D'], errors='coerce')
df = df[(df['Year'] >= 1924) & (df['Year'] <= 2024)]

# Compute average temperature anomaly per decade
df['Decade'] = (df['Year'] // 10) * 10  # e.g., 1924 → 1920s
decade_means = df.groupby('Decade')['J-D'].mean().reset_index()

# Create interactive bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=decade_means['Decade'].astype(str),
    y=decade_means['J-D'],
    marker=dict(color='green'),
    hovertemplate=(
        "Decade: %{x}s<br>"
        "Mean Anomaly: %{y:.2f} °C<extra></extra>"
    ),
    name="Decade Mean"
))

# Add value labels above bars
fig.update_traces(
    text=[f"{v:.2f}" for v in decade_means["J-D"]],
    textposition="outside"
)

# Layout and styling
fig.update_layout(
    title="Average Global Temperature Change per Decade (1924–2024)",
    xaxis_title="Decade",
    yaxis_title="Mean Temperature Anomaly (°C)",
    template="simple_white",
    width=950,
    height=550,
    margin=dict(l=60, r=40, t=80, b=60),
    xaxis=dict(tickangle=45)
)
fig.write_html("../plots/BarChart.html")
fig.show()

## Viz2

In [51]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

# Load data
df = pd.read_csv("../data/NASA_Data.csv", skiprows=1)
df = df[["Year", "J-D"]].dropna()
df["J-D"] = pd.to_numeric(df["J-D"], errors="coerce")
df = df[(df["Year"] >= 1924) & (df["Year"] <= 2024)]

years = df["Year"]
anoms = df["J-D"]

# Colorscale setup
colors = px.colors.diverging.RdBu_r
norm = (anoms - anoms.min()) / (anoms.max() - anoms.min())
color_vals = [colors[int(v * (len(colors) - 1))] for v in norm]

# Add decade column for dropdown interaction
df["Decade"] = (df["Year"] // 10) * 10

# Build base figure
fig = go.Figure()

# Main temperature anomaly line
fig.add_trace(go.Scatter(
    x=years,
    y=anoms,
    mode='lines',
    line=dict(color='black', width=2),
    hoverinfo='skip',
    name="Temperature Anomaly"
))

# Colored annual points
fig.add_trace(go.Scatter(
    x=years,
    y=anoms,
    mode='markers',
    marker=dict(
        size=8,
        color=color_vals,
        colorbar=dict(),
        showscale=True
    ),
    text=[f"Year: {y}<br>Anomaly: {a} °C" for y, a in zip(years, anoms)],
    hovertemplate="%{text}<extra></extra>",
    name="Annual Data"
))

# Zero line
fig.add_hline(
    y=0,
    line_width=2,
    line_dash="dash",
    line_color="gray"
)

# Decade dropdown interaction
dropdown_buttons = [
    dict(
        label="All Years",
        method="update",
        args=[
            {"visible": [True, True]},  # show both traces
            {"title": "Annual Global Temperature Anomalies (1924–2024)"}
        ]
    )
]

for decade in sorted(df["Decade"].unique()):
    mask = df["Decade"] == decade

    dropdown_buttons.append(
        dict(
            label=f"{decade}s",
            method="update",
            args=[
                {
                    "x": [years, years[mask]],
                    "y": [anoms, anoms[mask]],
                },
                {"title": f"Annual Global Temperature Anomalies — {decade}s"}
            ]
        )
    )

fig.update_layout(
    updatemenus=[dict(
        type="dropdown",
        showactive=True,
        buttons=dropdown_buttons,
        x=1.12,
        y=1.05
    )]
)

fig.update_layout(
    title="Annual Global Temperature Anomalies (1924–2024)<br><sup>Interactive 2D Line Visualization</sup>",
    template="simple_white",
    width=1150,
    height=600,
    margin=dict(l=60, r=80, t=110, b=60),
    xaxis=dict(
        title="Year",
        rangeslider=dict(visible=True),
        showgrid=False
    ),
    font=dict(size=14)
)
fig.write_html("../plots/NASA_annomalies.html")
fig.show()

## viz-3

In [52]:
import pandas as pd
import altair as alt
import webbrowser, os


def open_html(filename):
    path = os.path.abspath(filename)
    webbrowser.open("file://" + path)

# ============================
#  LOAD NOAA DATA
# ============================
noaa = pd.read_csv("../data/NOAA_GlobalData.csv", comment="#")
noaa = noaa[(noaa["Year"] >= 1924) & (noaa["Year"] <= 2024)]
noaa = noaa.sort_values("Year")
noaa["Rolling_10yr"] = noaa["Anomaly"].rolling(window=10, center=True).mean()

# ============================
#  LOAD BERKELEY MONTHLY DATA
# ============================
berk = pd.read_csv("../data/monthly.csv")
berk["Date"] = pd.to_datetime(berk["Year"], format="%Y-%m")
berk["Month"] = berk["Date"].dt.month_name()
berk["Year_num"] = berk["Date"].dt.year
berk = berk[(berk["Year_num"] >= 1924) & (berk["Year_num"] <= 2024)]


## Viz-4
 

In [53]:
# ======================================================
#  VISUALIZATION 1 (NOAA TREND WITH ROLLING AVERAGE)
# ======================================================

selector1 = alt.selection_point(
    fields=["Year"], on="mouseover", nearest=True, empty="none"
)

line = alt.Chart(noaa).mark_line(color="#1a5e2a").encode(
    x="Year:Q",
    y="Anomaly:Q"
)

smooth = alt.Chart(noaa).mark_line(
    color="#4CAF50", strokeWidth=3
).encode(
    x="Year:Q",
    y="Rolling_10yr:Q"
)

points = line.mark_circle(color="#4CAF50").encode(
    opacity=alt.condition(selector1, alt.value(1), alt.value(0))
)

rule = alt.Chart(noaa).mark_rule(color="#2e7d32").encode(
    x="Year:Q"
).transform_filter(selector1)

vis1 = (line + smooth + points + rule).add_params(selector1).properties(
    title="NOAA Global Temperature Anomalies (1924–2024, Interactive)",
    width=650,
    height=350
)

vis1_path = "../plots/NOAA_annomalies.html"
vis1.save(vis1_path)

vis1.show()

alt.LayerChart(...)

In [ ]:
# ======================================
#  VISUALIZATION 2 (NOAA DECADE BOXPLOT)
# ======================================

noaa["Decade"] = (noaa["Year"] // 10) * 10
selector2 = alt.selection_point(fields=["Decade"], bind="legend")

vis2 = alt.Chart(noaa).mark_boxplot(color="#388E3C").encode(
    x=alt.X("Decade:N", title="Decade"),
    y=alt.Y("Anomaly:Q", title="Temperature Anomaly (°C)"),
    color=alt.Color("Decade:N", scale=alt.Scale(scheme="greens"))
).add_params(selector2).properties(
    title="Distribution of NOAA Temperature Anomalies by Decade (Interactive)",
    width=650,
    height=350
)

vis2_path = "../plots/boxplot.html"
vis2.save(vis2_path)
vis2.show()

alt.Chart(...)

In [56]:
# ======================================
#  VISUALIZATION 3 (BERKELEY SEASONAL HEATMAP)
# ======================================

vis3 = alt.Chart(berk).mark_rect().encode(
    x=alt.X("Year_num:O", title="Year"),
    y=alt.Y(
        "Month:N",
        sort=[
            "January", "February", "March", "April", "May", "June",
            "July", "August", "September", "October", "November", "December"
        ]
    ),
    color=alt.Color(
        "Mean:Q",
        title="Temp Anomaly (°C)",
        scale=alt.Scale(scheme="greens")
    )
).properties(
    title="Berkeley Earth Monthly Temperature Anomalies (1924–2024)",
    width=650,
    height=350
)

vis3_path = "../plots/heatmap.html"
vis3.save(vis3_path)

vis3.show()


alt.Chart(...)